In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'

In [ ]:
import os

from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import scipy.sparse

import anndata
import scanpy as sc
import datasets
from datasets import Dataset

import torch
from scmg.model.contrastive_embedding import (CellEmbedder, 
                         embed_adata)

In [ ]:
# Load the autoencoder model
model_ce_path = '../contrastive_embedding/trained_embedder/'

model_ce = torch.load(os.path.join(model_ce_path, 'model.pt'))
model_ce.load_state_dict(torch.load(os.path.join(model_ce_path, 'best_state_dict.pth')))

device = 'cuda:0'
model_ce.to(device)
model_ce.eval()

In [ ]:
dataset_names = [
    'standard_adata_AllenBrain_WB_MM_2023_all',
    'standard_adata_Allen_BrainAging_MM_2022_all',
    'standard_adata_Arutyunyan_Placenta_HS_2023_all',
    'standard_adata_Bhaduri_CtxDev_HS_2021_all',
    'standard_adata_Cao_dev_HS_2020_all',
    'standard_adata_Conde_Immune_HS_2022_all',
    'standard_adata_Cowan_Retina_HS_2020_fovea',
    'standard_adata_Cowan_Retina_HS_2020_periphery',
    'standard_adata_Deng_CarT_HS_2020_all',
    'standard_adata_Elmentaite_intestine_HS_2021_all',
    'standard_adata_Enge_Pancrea_HS_2017_all',
    'standard_adata_Eraslan_MultiTissue_HS_2022_all',
    'standard_adata_Fawkner-Corbett_IntestineDev_HS_2021_all',
    'standard_adata_Han_HS_2020_all',
    'standard_adata_He_LungDev_HS_2022_all',
    'standard_adata_Hrovatin_Pancrea_MM_2022_all',
    'standard_adata_Jardine_BloodDev_HS_2021_normal',
    'standard_adata_Khaled_Breast_HS_2023_all',
    'standard_adata_Kuppe_Heart_HS_2022_all',
    'standard_adata_LaManno_WBDev_MM_2021_all',
    'standard_adata_Lake_Kidney_HS_2023_all',
    'standard_adata_Lengyel_FallopianTube_HS_2022_all',
    'standard_adata_Litvinukova_Heart_HS_2020_all',
    'standard_adata_Park_Thymus_HS_2020_all',
    'standard_adata_Qiu_Organogenesis_MM_2022_all',
    'standard_adata_Qiu_whole_embryo_dev_MM_2024_all',
    'standard_adata_Sikkema_Lung_HS_2023_core',
    'standard_adata_Streets_Adipose_HS_2023_all',
    'standard_adata_Suo_ImmuneDev_HS_2022_all',
    'standard_adata_Tabula_Muris_MM_2020_10x',
    'standard_adata_Tabula_Muris_MM_2020_smart-seq',
    'standard_adata_Tabula_Sapiens_HS_2022_all',
    'standard_adata_Tyser_Embryo_HS_2021_all',
    'standard_adata_VentoTormo_Placenta_HS_2018_all',
    'standard_adata_Wiedemann_Skin_HS_2023_all',
    'standard_adata_Yanagida_Blastocyst_HS_2021_all',
    'standard_adata_Yu_MultiTissue_HS_2021_all'
 ]

import json
with open('dataset_cell_types.json', 'r') as f:
    ds_ct_dict = json.load(f)

In [ ]:
standard_gene_df = pd.read_csv(
    '/GPUData_xingjie/Softwares/SCMG_dev/scmg/data/standard_genes.csv')
standard_ids = list(standard_gene_df['human_id'])

adata_input_path = '/GPUData_xingjie/SCMG/sc_rna_data/'

In [ ]:
adata_ce_list = []

for ds in ds_ct_dict:
    dataset_name = f'standard_adata_{ds}'.replace(':', '_')
    if dataset_name not in dataset_names:
        continue

    print(dataset_name)
    adata = sc.read_h5ad(os.path.join(adata_input_path, f'{dataset_name}.h5ad'))
    display(adata)

    # Choose cell types to keep
    adata = adata[adata.obs['cell_type'].isin(ds_ct_dict[ds])]

    # Subset to the standard genes
    common_genes = np.intersect1d(standard_ids, adata.var['human_gene_id'])
    adata = adata[:, adata.var['human_gene_id'].isin(common_genes)].copy()
    adata.var.index = adata.var['human_gene_id']
    sc.pp.filter_cells(adata, min_genes=100)
    display(adata)

    # Extract seed cells for each cell type
    cell_types = list(adata.obs['cell_type'].value_counts().index)
    selected_cells = []
    for ct in cell_types:
        ct_cell_ids = list(adata.obs.index[adata.obs['cell_type'] == ct])

        if len(ct_cell_ids) > 100:
            ct_cell_ids = list(np.random.choice(ct_cell_ids, 100, replace=False))

        selected_cells.extend(ct_cell_ids)

    adata_selected = adata[selected_cells].copy()
    embed_adata(model_ce, adata_selected)

    local_adata_ce = anndata.AnnData(
        X=adata_selected.obsm['X_ce_latent'],
        obs=adata_selected.obs.copy()
    )
    adata_ce_list.append(local_adata_ce)

adata_ce = anndata.concat(adata_ce_list)
adata_ce.write_h5ad('ref_cell_adata.h5ad')

# Generate the reference UMAP

In [ ]:
ref_cell_emb_adata = sc.read_h5ad('ref_cell_adata.h5ad')
ref_cell_emb_adata

In [ ]:
# Map cell types to major cell types
major_ct_df = pd.read_csv('../cell_type_analysis/major_cell_type_annotation.csv')
ct_to_mct_map = {row['cell_type']: row['major_cell_type'] 
                for _, row in major_ct_df.iterrows()}

ref_cell_emb_adata.obs['major_cell_type'] = ref_cell_emb_adata.obs[
                                        'cell_type'].map(ct_to_mct_map)
ref_cell_emb_adata

In [ ]:
sc.pp.neighbors(ref_cell_emb_adata, use_rep='X', n_neighbors=50)

sc.tl.paga(ref_cell_emb_adata, groups='cell_type')
sc.pl.paga(ref_cell_emb_adata, plot=False)  
sc.tl.umap(ref_cell_emb_adata, init_pos='paga')

#sc.tl.umap(ref_cell_emb_adata)

In [ ]:
sc.pl.umap(ref_cell_emb_adata, color='dataset_id')
sc.pl.umap(ref_cell_emb_adata, color='major_cell_type')
sc.pl.umap(ref_cell_emb_adata, color='major_cell_type', legend_loc='on data', 
           legend_fontsize=6)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20, 20), dpi=200)
sc.pl.umap(ref_cell_emb_adata, color='cell_type', palette='tab20',
           legend_loc='on data', legend_fontsize=2, ax=ax, s=10)

In [ ]:
# Save the adata with the UMAP
ref_cell_emb_adata.write_h5ad('ref_cell_adata.h5ad')